In [3]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
import folium

In [4]:
distance_df = pd.read_csv('../data/osrm_distance_matrix.csv', index_col=0)

In [5]:
distance_matrix = distance_df.values

In [6]:
locations = list(distance_df.index)
locations

['Nochchiyagama',
 'Medawachchiya',
 'Medirigiriya',
 'Dambulla',
 'Chilaw',
 'Walipennagahamula',
 'Wellawa',
 'Polpithigama',
 'Maho',
 'Galgamuwa',
 'Alawwa',
 'Narammala',
 'Ibbagamuwa',
 'Katupotha',
 'Pothuhera',
 'Polpitigama',
 'Ikiriyagolla',
 'Anuradapura',
 'Aralaganwila',
 'Thirappane',
 'Dewahuwa',
 'Kumbukgate',
 'Kithalawa',
 'Polgahawela',
 'Siyambalagaskotuwa',
 'Minuwangate',
 'Rambawa',
 'Kurunagala',
 'Kiribathkumbura']

In [7]:
# Display the matrix
print("Distance Matrix (km):")
print(distance_matrix)

print("\nLocations:")
print(locations)

Distance Matrix (km):
[[  0.      53.5289 117.1457  76.1327 102.9948 132.6988 103.9614  70.8661
   62.8791  39.612  131.9956 111.311  116.6007  98.5076 113.1894  71.1478
  131.9508  31.3265 157.3869  45.4115  73.6527  87.7496 116.8822 123.2267
  103.262   73.9666  72.0033 103.8198 145.0332]
 [ 53.5289   0.     109.7074  79.4945 157.0729 169.4899 140.767   95.8095
   96.4586  73.1916 163.2395 144.8905 123.1707 132.0871 144.4334  96.0912
  156.1096  25.0779 149.9486  37.9732  87.935  125.6694 150.4617 154.4707
  136.8415 107.5461  96.9467 135.9846 163.0233]
 [117.1457 109.7074   0.      57.749  187.2679 169.9369 119.0216 102.7374
  113.4241 106.7332 141.4941 132.9629 101.4253 140.3003 122.688  103.0191
  134.3642  92.3079  60.8041  71.743   72.3273 103.924  145.5504 132.7253
  142.6954 145.6764 103.8745 114.2391 141.2778]
 [ 76.1327  79.4945  57.749    0.     129.6105 112.2795  61.3642  45.08
   61.1097  68.9922  83.8367  75.3055  43.7679  82.6429  65.0306  45.3616
   76.7068  62.0714  9

In [8]:
depot = 'Kiribathkumbura'
depot_index = locations.index(depot)
depot_index

28

In [9]:
# Create Routing Index Manager
manager = pywrapcp.RoutingIndexManager(
    len(distance_matrix), 1, depot_index
)  # Single vehicle, depot at index 0

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)


# Distance callback function
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix[from_node][to_node])


# Register the callback
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Search Parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Solve
solution = routing.SolveWithParameters(search_parameters)

# Print solution with total distance
if solution:
    print("\nOptimal Route:")
    index = routing.Start(0)
    total_distance = 0
    route_order = []

    while not routing.IsEnd(index):
        route_order.append(locations[manager.IndexToNode(index)])
        next_index = solution.Value(routing.NextVar(index))
        total_distance += distance_matrix[manager.IndexToNode(index)][
            manager.IndexToNode(next_index)
        ]
        index = next_index

    route_order.append(locations[manager.IndexToNode(index)])  # Return to depot

    print(" -> ".join(route_order))
    print(f"Total Distance: {total_distance:.2f} km")
else:
    print("No solution found.")


Optimal Route:
Kiribathkumbura -> Ikiriyagolla -> Polgahawela -> Alawwa -> Kithalawa -> Walipennagahamula -> Chilaw -> Katupotha -> Siyambalagaskotuwa -> Narammala -> Pothuhera -> Kurunagala -> Wellawa -> Kumbukgate -> Rambawa -> Polpitigama -> Polpithigama -> Maho -> Minuwangate -> Galgamuwa -> Nochchiyagama -> Anuradapura -> Medawachchiya -> Thirappane -> Aralaganwila -> Medirigiriya -> Dambulla -> Dewahuwa -> Ibbagamuwa -> Kiribathkumbura
Total Distance: 812.61 km


In [9]:
gps_df = pd.read_csv('../data/data_gps.csv')

In [10]:
unique_locations = gps_df.drop_duplicates(subset=["ADDRESS"])[
    ["ADDRESS", "LATITUDE", "LONGITUDE"]
]

In [11]:
unique_locations.iloc[1]["ADDRESS"]

'Medawachchiya'

In [12]:
towns = {
    unique_locations.iloc[i]["ADDRESS"]: [
        unique_locations.iloc[i]["LATITUDE"],
        unique_locations.iloc[i]["LONGITUDE"],
    ]
    for i in range(len(unique_locations))
}

In [13]:
towns['Kiribathkumbura'] = [7.2726897, 80.5618714]
towns

{'Nochchiyagama': [np.float64(8.2640142), np.float64(80.2105099)],
 'Medawachchiya': [np.float64(8.5387775), np.float64(80.492996)],
 'Medirigiriya': [np.float64(8.1423253), np.float64(80.971342)],
 'Dambulla': [np.float64(7.8742031), np.float64(80.6510917)],
 'Chilaw': [np.float64(7.5765074), np.float64(79.7956755)],
 'Walipennagahamula': [np.float64(7.4307776), np.float64(79.9298775)],
 'Wellawa': [np.float64(7.5581214), np.float64(80.3664063)],
 'Polpithigama': [np.float64(7.8172167), np.float64(80.4049605)],
 'Maho': [np.float64(7.8239143), np.float64(80.273567)],
 'Galgamuwa': [np.float64(7.9956682), np.float64(80.2706117)],
 'Alawwa': [np.float64(7.2933377), np.float64(80.2387243)],
 'Narammala': [np.float64(7.43073), np.float64(80.2143788)],
 'Ibbagamuwa': [np.float64(7.5479128), np.float64(80.4496417)],
 'Katupotha': [np.float64(7.53362), np.float64(80.1875737)],
 'Pothuhera': [np.float64(7.4198002), np.float64(80.3282461)],
 'Polpitigama': [np.float64(7.8204405), np.float64(80

In [19]:
# Function to fetch actual road path using OSRM
def get_actual_road_path(town_coords):
    """
    Fetch the actual road path from OSRM API.
    :param town_coords: List of coordinates for the towns in the route.
    :return: GeoJSON road path coordinates.
    """
    # Format coordinates for OSRM API
    coords = ";".join([f"{lon},{lat}" for lat, lon in town_coords])

    # OSRM API URL
    url = f"http://router.project-osrm.org/route/v1/car/{coords}?overview=full&geometries=geojson"

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            coordinates = data["routes"][0]["geometry"]["coordinates"]
            print(coordinates)
            return coordinates
    return None


# Visualize routes
def visualize_routes(route, towns):
    """
    Visualize the actual road path on a map.
    :param route: List of town names in the route.
    :param towns: Dictionary of town names and their coordinates.
    :return: Folium map object.
    """
    # Create a base map centered around Kiribathkumbura
    kiribathkumbura_coords = towns["Nochchiyagama"]
    folium_map = folium.Map(location=kiribathkumbura_coords, zoom_start=10)

    # Get coordinates for the towns in the route
    route_coords = [towns[town] for town in route]
    

    # Fetch the actual road path
    road_path = get_actual_road_path(route_coords)
    print(road_path)

    # Plot the road path on the map
    if road_path:
        folium.PolyLine(
            [(lat, lon) for lon, lat in road_path], color="blue", weight=5, opacity=0.7
        ).add_to(folium_map)
    else:
        print("Could not fetch road path from OSRM.")

    # Add markers for each town
    for town in route:
        coord = towns[town]
        folium.Marker(location=coord, popup=town).add_to(folium_map)

    return folium_map

In [20]:
route_order

['Kiribathkumbura',
 'Ikiriyagolla',
 'Pothuhera',
 'Kurunagala',
 'Wellawa',
 'Ibbagamuwa',
 'Kumbukgate',
 'Rambawa',
 'Polpithigama',
 'Polpitigama',
 'Dewahuwa',
 'Dambulla',
 'Medirigiriya',
 'Aralaganwila',
 'Thirappane',
 'Medawachchiya',
 'Anuradapura',
 'Nochchiyagama',
 'Galgamuwa',
 'Maho',
 'Minuwangate',
 'Chilaw',
 'Walipennagahamula',
 'Kithalawa',
 'Siyambalagaskotuwa',
 'Katupotha',
 'Narammala',
 'Alawwa',
 'Polgahawela',
 'Kiribathkumbura']

In [21]:
# Visualize the route
map_object = visualize_routes(route_order, towns)

[[80.561653, 7.272856], [80.561528, 7.272694], [80.561245, 7.272414], [80.560926, 7.272193], [80.560854, 7.272207], [80.560754, 7.272275], [80.560637, 7.27223], [80.560289, 7.271856], [80.560252, 7.271527], [80.560161, 7.271295], [80.559921, 7.270865], [80.559612, 7.270831], [80.559361, 7.270848], [80.558577, 7.270729], [80.558434, 7.270626], [80.5581, 7.270442], [80.556765, 7.269078], [80.556184, 7.268729], [80.555218, 7.268259], [80.553714, 7.267436], [80.55332, 7.267368], [80.55312, 7.267286], [80.553065, 7.267442], [80.552928, 7.267582], [80.552426, 7.266971], [80.552334, 7.266858], [80.552263, 7.266747], [80.552151, 7.266649], [80.551951, 7.266506], [80.551987, 7.266421], [80.552002, 7.266368], [80.552006, 7.266336], [80.552027, 7.266192], [80.552072, 7.265859], [80.552078, 7.265765], [80.552148, 7.265711], [80.552188, 7.265696], [80.552036, 7.265613], [80.550536, 7.26473], [80.550017, 7.26436], [80.549682, 7.264168], [80.548899, 7.263697], [80.548443, 7.263434], [80.548252, 7.263

In [22]:
map_object

In [17]:
map_object